Tools

In [1]:
from google.colab import drive
import sys
sys.path.insert(0,'/content/drive/My Drive/Códigos/Python/Optimización I/tarea9')


In [2]:
from benchmark import SoftNoise
from optimization import *
from report import get_graf, plot_imgs
import time
import matplotlib.pyplot as plt
import cv2
import pandas as pd

Parámetros

In [14]:
# Parámetros función:

org_path = r'/content/drive/My Drive/Códigos/Python/Optimización I/tarea9/lena.png'
org_img = cv2.imread(org_path, 0) # Imagen que se utilizará como función "g"
lambdas = [0.1, 1., 10., 100.]  # Parámetros de regularización
mu = 0.01

# Parámetros punto inicial:
noise_path = r'/content/drive/My Drive/Códigos/Python/Optimización I/tarea9/lenanoise.png'  # Dirección de la imagen
noise_img = cv2.imread(noise_path, 0)  # Lectura de imagen en escala de grises
x_0 = noise_img.flatten()  # Punto inicial

# Parámetros algoritmo de optimización
betas =  ['FR', 'PR', 'HS', 'FRPR']   # Cálculo de beta en gradiente conjugado no lineal
c_1, c_2 = 1e-4, 0.95  # búsquesa lineal con bisección y condiciones fuertes de wolfe

# Parámetros criterios de paro:
tol = 1.e-2  # Tolerancia norma del gradiente
max_iter = 200  # Número máximo de iteraciones

# Parámetros estadísticas
filename_stats = '/content/drive/My Drive/Códigos/Python/Optimización I/tarea9/estadísticas.csv'
nruns = 1
data = []
index = []
x_target = org_img.flatten()

# Parámetros gráficas
start_plot = 3
filename_new_img = r'/content/drive/My Drive/Códigos/Python/Optimización I/tarea9/new_img.png'


 Estadísticas, gŕaficas e imágenes resultantes de gradiente conjugado no lineal

In [15]:
for lmbda in lambdas:
  for beta in betas:

    # Construcción de función objetivo y gradiente
    test_function = SoftNoise(org_img, lmbda, mu) 

    # Construcción de funciones para variante del método
    update_alpha = bisection_with_wolfe(c_1, c_2, test_function)
    method_name = "beta = " + beta
    print("-"*100)
    print('\n Optimización con '+method_name + ', lambda = '+str(lmbda))
    update_beta = get_non_linear_beta(beta)

    # Estadísticas
    times = []
    iterations = []
    errors = []

    for run in range(nruns):
        # Optimización
        begin = time.time()
        x, f, g, test_function, k = conjugate_gradient_non_linear(x_0, test_function, tol, update_beta,
                                                                  max_iter, update_alpha)
        time.sleep(1)
        end = time.time()
        times += [end - begin]
        iterations += [k]
        dif = x_target - x
        err = np.linalg.norm(dif)
        errors += [err]

    # Gráficas
    get_graf(test_function, start_plot)


    # Imagen suavizada
    new_img = x.reshape(org_img.shape)
    plot_imgs(org_img, noise_img, new_img)


    # Tabla con reporte de desempeño:

    method_dict = {"Método": beta,
                    "Tiempo promedio (s)": round(np.mean(times), 2),
                    "Iteraciones promedio": round(np.mean(iterations), 2),
                    "Error promedio": round(np.mean(errors), 2),
                    "Lambda": round(lmbda, 2)
                    }

    data.append(method_dict)
    index.append(beta)


stats = pd.DataFrame(data, index=index)
print(stats.to_string(index=False))


Output hidden; open in https://colab.research.google.com to view.

In [16]:


stats.to_csv(filename_stats, encoding='utf-8', index=False)
